In [ ]:
from cell import Cell
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
def objective(bounds, env):
    cell = Cell(*bounds, *env)
    fitness = -1 * cell.calc_fitness()

    return fitness

In [ ]:
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

In [ ]:
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = np.random.randint(len(pop))
	for ix in np.random.randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [ ]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if np.random.rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = np.random.randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [ ]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if np.random.rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [ ]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut, env_settings, env_n):
	# initial population of random bitstring
	pop = [np.random.randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]), env_settings)

	avg = []
	std = []

	data = {}

	# enumerate generations
	for gen in range(n_iter):
		print(f'> gen: {gen}')
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d, env_settings) for d in decoded]

		data.update({str(gen) + '_scores' : scores})
		data.update({str(gen) + '_decoded' : decoded})

		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(f'> gen: {gen}, new best: f({decoded[i]}) = {-1 * scores[i]}')
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = []
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children

		avg.append(sum(scores)/len(scores))
		std.append(np.std(scores))
		
		print(f'> gen: {gen}, avg = {-1 * avg[gen]}, std = {std[gen]}, best = {-1 * best_eval}')
	return [best, best_eval, data]


In [47]:
bounds = [[0.001,1.0]] * 3
#env_settings = [.1,.1,.8,.9,.8,.35,200] # optimal p
env_settings = [.6,.6,.8,.9,.1,.65,200] # optimal c

n_iter = 100
n_bits = 16
n_pop = 100
r_cross = 0.9
r_mut = 1.0 / (float(n_bits) * len(bounds))

n_bits = 16
r_mut = 1.0 / (float(n_bits) * len(bounds))
n_pop = 100

env_n = 1

In [48]:
best, score, data = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut, env_settings, env_n)
print('done')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, -1 * score))

> gen: 0
> gen: 0, new best: f([0.014063705444335939, 0.7528109130859375, 0.2977458038330078]) = 288.2889391662601
> gen: 0, new best: f([0.6747030639648437, 0.8055077972412109, 0.6908612060546875]) = 737.6039125288999
> gen: 0, new best: f([0.032858978271484374, 0.9551535339355469, 0.566108154296875]) = 772.4755821812234
> gen: 0, new best: f([0.1393350372314453, 0.9460836334228515, 0.7248237915039063]) = 779.9355608019072
> gen: 0, avg = 244.61637742285416, std = 183.87046088754022, best = 779.9355608019072
> gen: 1
> gen: 1, new best: f([0.6115186309814453, 0.9460836334228515, 0.7238482055664063]) = 802.3482389888225
> gen: 1, new best: f([0.48147607421875, 0.9493152618408203, 0.6941690521240235]) = 861.3302826780355
> gen: 1, new best: f([0.1393350372314453, 0.9463275299072266, 0.6883612670898438]) = 867.0471237314886
> gen: 1, new best: f([0.9566626434326172, 0.9647874450683593, 0.6897331848144531]) = 870.6379024789035
> gen: 1, avg = 432.53975724556165, std = 221.6024413294299, b

In [49]:
df = pd.DataFrame(data=data)
df.head()

,0_scores,0_decoded,1_scores,1_decoded,2_scores,2_decoded,3_scores,3_decoded,4_scores,4_decoded,...,95_scores,95_decoded,96_scores,96_decoded,97_scores,97_decoded,98_scores,98_decoded,99_scores,99_decoded
0,-205.495231,"[0.5836077270507812, 0.5652850036621093, 0.901...",-310.633655,"[0.5705592651367187, 0.8251109771728515, 0.303...",-712.385970,"[0.17532501220703126, 0.805751693725586, 0.690...",-668.141909,"[0.6876600646972656, 0.7971695861816406, 0.566...",-836.937592,"[0.23178704833984376, 0.9496811065673828, 0.69...",...,-920.998544,"[0.7246865997314453, 0.9999847564697265, 0.654...",-944.898481,"[0.19299226379394532, 0.9995731811523437, 0.64...",-917.857381,"[0.7253573150634766, 0.9996189117431641, 0.654...",-921.323174,"[0.5515200958251953, 0.9991158752441406, 0.652...",-920.122439,"[0.544264175415039, 0.9997256164550781, 0.6500..."
1,-288.288939,"[0.014063705444335939, 0.7528109130859375, 0.2...",-347.577738,"[0.9556870574951172, 0.9610222930908203, 0.966...",-863.615093,"[0.9566626434326172, 0.9647874450683593, 0.689...",-498.393422,"[0.9228372497558593, 0.5900404968261719, 0.722...",-791.499435,"[0.3562352294921875, 0.8790730743408203, 0.691...",...,-945.944323,"[0.7207842559814454, 0.9999542694091796, 0.654...",-919.285696,"[0.6110308380126953, 0.9968141021728516, 0.650...",-667.829285,"[0.8173672637939453, 0.999862808227539, 0.5293...",-927.613943,"[0.5985311431884766, 0.999862808227539, 0.6485...",-907.173938,"[0.567678237915039, 0.9956708374023437, 0.6506..."
2,-737.603913,"[0.6747030639648437, 0.8055077972412109, 0.690...",-430.247302,"[0.504158447265625, 0.48972282409667967, 0.694...",-722.175522,"[0.23776251220703126, 0.805751693725586, 0.690...",-784.967389,"[0.48147607421875, 0.9492847747802734, 0.56598...",-851.919491,"[0.3540858917236328, 0.9921038513183593, 0.705...",...,-374.607868,"[0.5597973327636718, 0.9999542694091796, 0.902...",-933.919873,"[0.5334717559814454, 0.9999542694091796, 0.654...",-937.509775,"[0.22378419494628907, 0.9968141021728516, 0.63...",-902.877697,"[0.8339522247314453, 0.984009536743164, 0.6263...",-924.786973,"[0.5673733673095703, 0.9998170776367188, 0.648..."
3,-543.180527,"[0.37624998474121096, 0.6382252960205078, 0.71...",-347.506208,"[0.8392417297363282, 0.5716720428466797, 0.789...",-164.717147,"[0.18658998107910157, 0.3785974884033203, 0.82...",-679.767784,"[0.6204208526611328, 0.8212086334228516, 0.724...",-869.504737,"[0.9686593017578125, 0.9628820037841797, 0.692...",...,-818.488501,"[0.7237110137939453, 0.9999542694091796, 0.717...",-950.841674,"[0.7197934265136718, 0.9999847564697265, 0.648...",-361.571921,"[0.6111527862548828, 0.9971647033691406, 0.156...",-915.324198,"[0.9887655181884766, 0.999862808227539, 0.6542...",-927.217226,"[0.6462129058837891, 0.9997408599853516, 0.645..."
4,-197.103449,"[0.492451416015625, 0.37742373657226563, 0.803...",-334.969946,"[0.9566626434326172, 0.8985847930908203, 0.300...",-840.283649,"[0.48147607421875, 0.9492847747802734, 0.69086...",-436.633162,"[0.9257487640380859, 0.48972282409667967, 0.69...",-762.997313,"[0.3540858917236328, 0.8653234100341797, 0.692...",...,-906.956302,"[0.5988817443847656, 0.9834912567138672, 0.648...",-923.063954,"[0.7899136657714844, 0.9999542694091796, 0.648...",-904.116249,"[0.19396784973144532, 0.9843601379394531, 0.64...",-939.449992,"[0.5751780548095703, 0.9999847564697265, 0.654...",-932.228256,"[0.614994155883789, 0.999862808227539, 0.64837..."


In [50]:
# df.to_csv(os.getcwd() + '\\data_optimal_p.csv')
df.to_csv(os.getcwd() + '\\data_optimal_c.csv')

In [ ]:
# create feature vectors:
# - (pigments, chemoenzymes)
# - (pigments, microvili)
# - (chemoenzymes, microvili)